# neuralcoref 사용법

In [1]:
import spacy
nlp = spacy.load('en')

import neuralcoref
neuralcoref.add_to_pipe(nlp)

doc = nlp(u'My sister has a dog. She loves him.')

print(doc._.has_coref)
print(doc._.coref_clusters)

True
[My sister: [My sister, She], a dog: [a dog, him]]


In [2]:
import spacy
nlp = spacy.load('en')

import neuralcoref
coref = neuralcoref.NeuralCoref(nlp.vocab)
nlp.add_pipe(coref, name='neuralcoref')

doc = nlp(u'My sister has a dog. She loves him.')

print(doc._.has_coref)
print(doc._.coref_clusters)

True
[My sister: [My sister, She], a dog: [a dog, him]]


In [3]:
print(f'doc은\n{doc}')
print(doc._.coref_resolved)
print(doc._.coref_scores)
print('')
span = doc[-7:-5]
print(f'span은\n{span}')

print(span._.is_coref)
print(span._.coref_cluster)
print(span._.coref_scores)
print('')
token = doc[-10]
print(f'token은\n{token}')

print(token._.in_coref)
print(token._.coref_clusters)

doc은
My sister has a dog. She loves him.
My sister has a dog. My sister loves a dog.
{My sister: {My sister: 1.3110305070877075}, a dog: {a dog: 1.804752230644226, My sister: -1.6715972423553467}, She: {She: -0.10834205150604248, My sister: 8.058426856994629, a dog: -1.0625176429748535}, him: {him: -1.870743989944458, My sister: 3.1147186756134033, a dog: 4.356405258178711, She: -3.1379528045654297}}

span은
a dog
True
a dog: [a dog, him]
{a dog: 1.804752230644226, My sister: -1.6715972423553467}

token은
My
True
[My sister: [My sister, She]]


In [4]:
import spacy
import neuralcoref

nlp = spacy.load('en')
neuralcoref.add_to_pipe(nlp)
doc = nlp(u'Deepika has a dog. She loves him. The movie star has always been fond of animals')
print(doc._.coref_clusters)
print(doc._.coref_resolved)

[Deepika: [Deepika, She, him, The movie star]]
Deepika has a dog. Deepika loves Deepika. Deepika has always been fond of animals


In [5]:
nlp.remove_pipe("neuralcoref")
neuralcoref.add_to_pipe(nlp, conv_dict={'Deepika':['woman','actress']})

doc = nlp(u'Deepika has a dog. She loves him. The movies star has always been fond of animals')
doc._.coref_clusters

[Deepika: [Deepika, She, The movies star], a dog: [a dog, him]]

In [6]:
doc._.coref_resolved

'Deepika has a dog. Deepika loves a dog. Deepika has always been fond of animals'

# Coreference Resolution

In [7]:
# spacy load하기
nlp = spacy.load('en')

# spacy pipe에 neural coref 추가하기
neuralcoref.add_to_pipe(nlp)

def coref_resolution(text):
    '''
    주어진 텍스트에 대해 coreference resolution 수행
    '''
    doc = nlp(text)
    return doc._.coref_resolved

text = '''
Elon Musk is a business magnate, industrial designer, and engineer. He is the founder, CEO, CTO, and chief designer of SpaceX. He is also early investor, CEO, and product architect of Tesla, Inc. He is also the founder of The Boring Company and the co-founder of Neuralink. A centibillionaire, Musk became the richest person in the world in January 2021, with an estimated net worth of $185 billion at the time, surpassing Jeff Bezos. Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa. He briefly attended the University of Pretoria before moving to Canada aged 17 to attend Queen's University. He transferred to the University of Pennsylvania two years later, where he received dual bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but decided instead to pursue a business career. He went on co-founding a web software company Zip2 with his brother Kimbal Musk.
'''

print(text)
print('------------------------------------------')
print(coref_resolution(text))
    


Elon Musk is a business magnate, industrial designer, and engineer. He is the founder, CEO, CTO, and chief designer of SpaceX. He is also early investor, CEO, and product architect of Tesla, Inc. He is also the founder of The Boring Company and the co-founder of Neuralink. A centibillionaire, Musk became the richest person in the world in January 2021, with an estimated net worth of $185 billion at the time, surpassing Jeff Bezos. Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa. He briefly attended the University of Pretoria before moving to Canada aged 17 to attend Queen's University. He transferred to the University of Pennsylvania two years later, where he received dual bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but decided instead to pursue a business career. He went on co-founding a web software company Zip2 with his brother Kimbal Musk.

------------------------------

# Named Entity Linking

In [8]:
import urllib
from string import punctuation
import nltk
import json

In [9]:
ENTITY_TYPES = ["human", "person", "company", "enterprise", "business", "geographic region",
                "human settlement", "geographic entity", "territorial entity type", "organization"]

def CallWikifier(text, lang="en", threshold=0.8):
    # Prepare the URL.
    data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "insert your user key here"),
        ("pageRankSqThreshold", "%g" % threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "200"), ("nWordsToIgnoreFromList", "200"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", "2"),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
        ])
    url = "http://www.wikifier.org/annotate-article"
    # Call the Wikifier and read the response.
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout = 60) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
    # Output the annotations.

    for annotation in response["annotations"]:
        print(annotation)

CallWikifier("Syria's foreign minister has said Damascus is ready")

KeyError: 'annotations'

In [10]:
import urllib
from string import punctuation
import nltk

ENTITY_TYPES = ["human", "person", "company", "enterprise", "business", "geographic region",
                "human settlement", "geographic entity", "territorial entity type", "organization"]

def wikifier(text, lang="en", threshold=0.8):
    """Function that fetches entity linking results from wikifier.com API"""
    # Prepare the URL.
    data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "tgbdmkpmkluegqfbawcwjywieevmza"),
        ("pageRankSqThreshold", "%g" %
         threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "100"), ("nWordsToIgnoreFromList", "100"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", "2"),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
    ])
    url = "http://www.wikifier.org/annotate-article"
    # Call the Wikifier and read the response.
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout=60) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
    # Output the annotations.
    results = list()
    for annotation in response["annotations"]:
        # Filter out desired entity classes
        if ('wikiDataClasses' in annotation) and (any([el['enLabel'] in ENTITY_TYPES for el in annotation['wikiDataClasses']])):

            # Specify entity label
            if any([el['enLabel'] in ["human", "person"] for el in annotation['wikiDataClasses']]):
                label = 'Person'
            elif any([el['enLabel'] in ["company", "enterprise", "business", "organization"] for el in annotation['wikiDataClasses']]):
                label = 'Organization'
            elif any([el['enLabel'] in ["geographic region", "human settlement", "geographic entity", "territorial entity type"] for el in annotation['wikiDataClasses']]):
                label = 'Location'
            else:
                label = None

            results.append({'title': annotation['title'], 'wikiId': annotation['wikiDataItemId'], 'label': label,
                            'characters': [(el['chFrom'], el['chTo']) for el in annotation['support']]})
    return results

In [11]:
# First get all the entities in the sentence
entities = wikifier(sentence, threshold=entities_threshold)
# Iterate over every permutation pair of entities
for permutation in itertools.permutations(entities, 2):
    for source in permutation[0]['characters']:
        for target in permutation[1]['characters']:
            # Relationship extraction with OpenNRE
            data = relation_model.infer(
                {'text': sentence, 'h': {'pos': [source[0], source[1] + 1]}, 't': {'pos': [target[0], target[1] + 1]}})
            if data[1] > relation_threshold:
                relations_list.append(
                    {'source': permutation[0]['title'], 'target': permutation[1]['title'], 'type': data[0]})


NameError: name 'sentence' is not defined